In [29]:
import pandas as pd
import pickle as pkl

import pybaseball as pb
from pybaseball import  playerid_lookup
from pybaseball import statcast
from pybaseball import statcast_batter
from pybaseball import statcast_pitcher
from pybaseball import batting_stats_range

import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [30]:
full_data = pb.statcast(start_dt='2018-03-29', end_dt='2018-10-28')

This is a large query, it may take a moment to complete


100%|██████████| 214/214 [04:48<00:00,  1.35s/it]


In [31]:
full_data

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
153,SL,2018-10-28,84.0,3.05,5.26,"Sale, Chris",592518,519242,strikeout,swinging_strike,...,5,5,1,1,5,Standard,Standard,281,-0.001,-0.067
159,FF,2018-10-28,95.3,3.17,5.5,"Sale, Chris",592518,519242,None,ball,...,5,5,1,1,5,Standard,Standard,147,0.0,0.008
162,FF,2018-10-28,96.4,3.07,5.54,"Sale, Chris",592518,519242,None,foul,...,5,5,1,1,5,Standard,Standard,135,0.0,-0.025
168,CH,2018-10-28,86.6,3.1,4.98,"Sale, Chris",592518,519242,None,swinging_strike,...,5,5,1,1,5,Standard,Standard,89,0.0,-0.016
173,SL,2018-10-28,78.5,3.31,5.16,"Sale, Chris",571771,519242,strikeout,swinging_strike,...,5,5,1,1,5,Standard,Standard,281,-0.005,-0.212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404,FF,2018-03-29,94.1,-2.01,6.43,"Archer, Chris",434670,502042,None,foul,...,0,0,0,0,0,Standard,Strategic,198,0.0,-0.016
3544,CH,2018-03-29,87.9,-2.43,6.27,"Archer, Chris",643217,502042,field_out,hit_into_play,...,0,0,0,0,0,Standard,Standard,220,0.015,-0.103
3662,FF,2018-03-29,94.8,-2.34,6.33,"Archer, Chris",643217,502042,None,foul,...,0,0,0,0,0,Standard,Standard,204,0.0,-0.037
3840,FF,2018-03-29,93.9,-2.29,6.41,"Archer, Chris",643217,502042,None,called_strike,...,0,0,0,0,0,Standard,Standard,193,0.0,-0.026


In [32]:
full_data = full_data[['pitch_type',
                     # 'game_date',
                     'release_speed', 'release_pos_x',
       'release_pos_z',
                     # 'player_name', 'batter', 'pitcher',
                     'events',
       'description', 
                     # 'spin_dir', 'spin_rate_deprecated',
       # 'break_angle_deprecated', 'break_length_deprecated', 
                     'zone', 
                     #'des',
       # 'game_type', 
                     'stand', 'p_throws',
                     # 'home_team', 'away_team',
                     'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 
                     # 'game_year', 
                     'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 
                     # 'on_3b', 'on_2b', 'on_1b',
       # 'outs_when_up', 'inning', 'inning_topbot',
                     'hc_x', 'hc_y',
       # 'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension',
                     # 'game_pk', 'pitcher.1',
       # 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       # 'fielder_7', 'fielder_8', 'fielder_9',
                     'release_pos_y',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'woba_value', 'woba_denom',
                     # 'babip_value', 'iso_value',
       'launch_speed_angle', 
                     # 'at_bat_number', 'pitch_number',
                     'pitch_name',
       # 'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score',
       # 'post_home_score', 'post_bat_score', 'post_fld_score',
       'if_fielding_alignment', 'of_fielding_alignment', 'spin_axis'#,
       # 'delta_home_win_exp', 'delta_run_exp'
                    ]].copy()

## Model Selection

### Features

In [33]:
X_full_data = full_data[['events', 'launch_angle', 'release_speed',  'hc_y', 'hc_x']].copy() # 'spin_axis', 'release_spin_rate', 'plate_z',  'zone', 'p_throws','stnad',
X_full_data.dropna(inplace=True)
X_full_data.reset_index()#inplace=True)
X_full_data

,events,launch_angle,release_speed,hc_y,hc_x
184,field_out,-21,93.2,184.03,143.9
193,grounded_into_double_play,2,92.8,151.58,146.44
186,home_run,23,86.7,44.11,58.23
195,field_out,69,95.8,211.4,113.49
204,field_out,37,87.8,45.63,120.58
...,...,...,...,...,...
3292,field_out,-12,88.9,149.62,146.83
3702,field_out,3,88.2,151.43,106.86
1316,field_out,-8,93.9,152.04,143.81
1766,field_out,-16,90.1,164.76,148.05


In [34]:
targets = ['single', 'field_out', 'home_run', 'walk', 'double', 'triple']
X_full_data = X_full_data[X_full_data['events'].isin(targets)].copy()

In [35]:
le = LabelEncoder()
X_full_data['events_encoded'] = le.fit_transform(X_full_data['events'])


In [36]:
X_full_data['events_encoded'].unique()

array([1, 2, 3, 4, 0])

In [37]:
X_full_data.dropna(inplace=True)
X_full_data.reset_index(inplace=True)

### Model

In [38]:
def score_dataset(X_train, X_valid, y_train, y_valid, model):
    model = model#87.9
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)

In [39]:
features = ['launch_angle', 'release_speed',  'hc_y', 'hc_x']
target = ['events_encoded']

In [40]:
folds = 10
kf = KFold(folds, shuffle=True)
accuracies = pd.Series()
s = pd.Series()
models = []
for train_idx, val_idx in kf.split(X_full_data.loc[:,features+target]):
    train = X_full_data.iloc[train_idx]
    val = X_full_data.iloc[val_idx]
    clf = XGBClassifier(random_state=3)
    clf.fit(train.loc[:, features], train.loc[:, target])
    y_pred = clf.predict(val.loc[:, features])#clf.predict_proba(X_val)[:, 1]
    accuracy = pd.Series(accuracy_score(val.loc[:, target], y_pred))
    accuracies = pd.concat([accuracies, accuracy])
    models.append(clf)

In [41]:
accuracies


0    0.879302
0    0.875833
0    0.877568
0    0.879850
0    0.877294
0    0.879576
0    0.877568
0    0.875194
0    0.878389
0    0.873094
dtype: float64

In [42]:
y = X_full_data['events_encoded']
X =X_full_data.drop(['events', 'events_encoded'], axis=1)

In [43]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=3)

In [44]:
score_dataset(X_train, X_valid, y_train, y_valid, models[2])

0.8748744636172738